## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Madang,PG,-5.2246,145.7966,74.89,94,100,5.53,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,75.42,73,56,12.91,broken clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,66.27,70,99,18.45,light rain
3,3,Punta Arenas,CL,-53.1500,-70.9167,53.71,76,75,18.41,light intensity shower rain
4,4,Luderitz,NaN,-26.6481,15.1594,64.65,75,2,9.73,clear sky


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cttemp_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cttemp_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Madang,PG,-5.2246,145.7966,74.89,94,100,5.53,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,75.42,73,56,12.91,broken clouds
5,5,Isangel,VU,-19.5500,169.2667,77.68,85,96,9.95,overcast clouds
7,7,Lodja,CD,-3.4833,23.4333,71.65,96,96,0.45,moderate rain
8,8,Keetmanshoop,NaN,-26.5833,18.1333,72.27,18,0,19.19,clear sky
9,9,Hilo,US,19.7297,-155.0900,74.28,98,100,3.00,moderate rain
13,13,Saint-Philippe,RE,-21.3585,55.7679,80.19,82,36,10.40,scattered clouds
16,16,Vilcun,CL,-38.6500,-72.2333,72.34,25,0,8.66,clear sky
18,18,Cosala,MX,24.4125,-106.6908,76.12,29,57,2.10,broken clouds
20,20,Wattegama,LK,6.7989,81.4808,74.43,93,17,1.74,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cttemp_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cttemp_df = preferred_cttemp_df.dropna()
clean_cttemp_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cttemp_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Madang,PG,74.89,overcast clouds,-5.2246,145.7966,
1,Rikitea,PF,75.42,broken clouds,-23.1203,-134.9692,
5,Isangel,VU,77.68,overcast clouds,-19.5500,169.2667,
7,Lodja,CD,71.65,moderate rain,-3.4833,23.4333,
9,Hilo,US,74.28,moderate rain,19.7297,-155.0900,
13,Saint-Philippe,RE,80.19,scattered clouds,-21.3585,55.7679,
16,Vilcun,CL,72.34,clear sky,-38.6500,-72.2333,
18,Cosala,MX,76.12,broken clouds,24.4125,-106.6908,
20,Wattegama,LK,74.43,few clouds,6.7989,81.4808,
22,Faanui,PF,80.40,light rain,-16.4833,-151.7500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_requests = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_requests["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan) 
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Madang,PG,74.89,overcast clouds,-5.2246,145.7966,Comfort Inn Hotel
1,Rikitea,PF,75.42,broken clouds,-23.1203,-134.9692,People ThankYou
5,Isangel,VU,77.68,overcast clouds,-19.5500,169.2667,Tanna Lodge
7,Lodja,CD,71.65,moderate rain,-3.4833,23.4333,
9,Hilo,US,74.28,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(20.0, 20.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,max_intensity=300, point_radius=3)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))